In [11]:
# the purpose of this script is pull infomation on all 4 csvs that are pandas data frames in one place. 
champ_api_key =input('champion.gg api key?')

champion.gg api key?RGAPI-a4df6fb2-f7bf-4c5c-a147-c7c74abf60b9


In [1]:
import cassiopeia as cs
import pandas as pd
import requests
import random
import time as t
from cassiopeia import riotapi

todaysdate = t.strftime('%Y-%m-%d')


In [2]:
riotkey= input('riot api?')

riot api?RGAPI-a4df6fb2-f7bf-4c5c-a147-c7c74abf60b9


In [3]:
riotapi.set_region("NA")
riotapi.set_api_key(riotkey)

In [ ]:

apicall= 'http://api.champion.gg/v2/champions?champData=kda,overallPerformanceScore&limit=200&api_key='+champ_api_key
champs = requests.get(apicall)


In [ ]:
champdf =pd.read_csv('championsstr.csv')

In [ ]:
champiter = len(champs.json()

In [ ]:
for num in range(champiter):
    kda = (champs.json()[num]['kills']+champs.json()[num]['assists'])/(champs.json()[num]['deaths'])
    indexval = champs.json()[num]['role']+str(champs.json()[num]['championId'])+todaysdate
    if indexval in champdf['indexval'].values:
        break
    single_champion_list= [{'K':champs.json()[num]['kills'], 'D':champs.json()[num]['deaths'], 'A':champs.json()[num]['assists'],  'CHAMPID_Role':str(champs.json()[num]['championId'])+champs.json()[num]['role'],'CHAMPID':str(champs.json()[num]['championId']),  'ROLE':champs.json()[num]['role'],'PATCH':champs.json()[num]['patch'],'WINRATE':champs.json()[num]['winRate'],'PLAYRATE':champs.json()[num]['playRate'],'RANK':champs.json()[num]['overallPerformanceScore'],'DATE':todaysdate, 'KDA': kda, 'indexval':indexval }]
    single_champion_dataframe = pd.DataFrame(single_champion_list)
    print(single_champion_dataframe.head())
    champdf= pd.concat([champdf,single_champion_dataframe])

In [ ]:
champdf.to_csv('championsstr.csv')


In [4]:
summoners_to_scrape = pd.read_csv('scrapetargets.csv', index_col='summoner_id' )
summonersdf = pd.read_csv('champskill.csv', index_col='summoner')


In [5]:
scrapelist = summoners_to_scrape.index

In [ ]:
for num in range(summoners_to_scrape.shape[0]):
    print('checking'+ str(scrapelist[num]))
    if scrapelist[num]  in summonersdf.index.values:
        print('skipping'+ str(scrapelist[num]))
        continue
    try:
        summoner = riotapi.get_summoner_by_id(int(scrapelist[num]))
        sumstats = riotapi.get_ranked_stats(summoner)
    except:
        continue
    rankedchampsnum = len(list(sumstats.keys()))

    summonerstatdict = {'summoner':summoner.id , 'summonername': summoner.name, 'date': todaysdate}
    print('gathering stats on summoner'+summoner.name)
    for champnum in range(rankedchampsnum):
        if list(sumstats.keys())[champnum] is None:
            champdata = sumstats[list(sumstats.keys())[champnum]].data
            summonerstatdict['AllK'] = champdata.averageChampionsKilled
            summonerstatdict['AllD'] = champdata.averageNumDeaths
            summonerstatdict['AllA'] = champdata.averageAssists
            summonerstatdict['AllW'] = champdata.totalSessionsWon
            summonerstatdict['AllL'] = champdata.totalSessionsLost
        else:
            champdata = sumstats[list(sumstats.keys())[champnum]].data
            #print(champnum)
            champstr = str(list(sumstats.keys())[champnum].id)
            #print(champstr)
            summonerstatdict[champstr+'K'] = champdata.averageChampionsKilled
            summonerstatdict[champstr+'D'] = champdata.averageNumDeaths
            summonerstatdict[champstr+'A'] = champdata.averageAssists
            summonerstatdict[champstr+'W'] = champdata.totalSessionsWon
            summonerstatdict[champstr+'L'] = champdata.totalSessionsLost
    summonerdf = pd.DataFrame([summonerstatdict], index=[summoner.id])
    # we have a dataframe that represents a single summoner's skill. 
    
    #we concatenate the current list of summoners and skills with ourselves.     
    summonersdf = pd.concat([summonersdf, summonerdf])
#we exit the loop before we write the csv
summonersdf.to_csv('champskill.csv')


In [6]:
summoners_to_scrape = pd.read_csv('scrapetargets.csv', index_col='summoner_id')
matchesdf = pd.read_csv('matches.csv')
scrapelist = summoners_to_scrape.index

In [99]:
scrapelist = summoners_to_scrape.index

In [7]:
# let's write a function that we will call in our larger loop to add summoner's to the scrapelist. 
def summoner_scrape_add(summonerid,summonername):
    global summoners_to_scrape
    if summonerid not in summoners_to_scrape.index.values:
        challenger= pd.DataFrame([{'summoner name': summonername, 'summoner_id'  :summonerid}], index=[summonerid])
        summoners_to_scrape =summoners_to_scrape.append(challenger )

In [70]:
summoner = riotapi.get_summoner_by_id(int(scrapelist[2]))

In [73]:
#Some manual testing to target the epoch I want. 
matchlist= riotapi.get_match_list(summoner, ranked_queues=['RANKED_SOLO_5X5', 'TEAM_BUILDER_RANKED_SOLO'], seasons='PRESEASON2017'  )
tempmatch =matchlist[0]
tempmatch.timestamp

datetime.datetime(2017, 7, 6, 6, 42, 4, 899000)

In [112]:
runtime = summoners_to_scrape.shape[0]
for iteration in range(runtime): # the outer loop for ever summoner currently on our watch list.
    # in earlier version of this code this went in order (for sumiter in range), but I've found that that makes the system re-run the same matches constantly. 
    #The code should be run every other time differently (sequentially vs randomly)
    sumiter = random.randint(0,runtime)
    try:
        summoner = riotapi.get_summoner_by_id(int(scrapelist[sumiter]))
        matchlist= riotapi.get_match_list(summoner, ranked_queues=['RANKED_SOLO_5X5', 'TEAM_BUILDER_RANKED_SOLO'], seasons='PRESEASON2017' )
    except KeyboardInterrupt:
                raise
    except:
        continue
# in case we get unlucky and a bad id is in the list, this skips the id and goes to the next one.  
    
    matchestoscrape = min(25,len(matchlist))
    for matchiter in range(matchestoscrape):
        #lets make sure we don't already know about this match
       
        if matchlist[matchiter].id in matchesdf.index.values:
            print('matchfound, on to the next')
            continue
        while True:
            print('new match... scraping..')
            try:
                match = riotapi.get_match(matchlist[matchiter])
                matchdata = riotapi.get_match(matchlist[matchiter], include_timeline=False).data
                break
            except KeyboardInterrupt:
                raise
            except:
                continue
                
        #lets worry about the globals (who won, when did it happen, the match id)
        team1win = matchdata.teams[0].winner
        team2win = matchdata.teams[1].winner
        matchid = matchlist[matchiter].id
        print('scraping '+str(matchid))
        matchtimestamp = matchlist[matchiter].timestamp
        matchversion = matchlist[matchiter].match().version
        # on to the interesting  part.  This is a pair of nested loops. The markdown cell contains the larger codeblock version, which may be more readable.
        matchdict = {}
        for playernum in range (10):
                matchdict['player'+str(playernum+1)+'champid']= matchdata.participants[playernum].championId
                matchdict['player'+str(playernum+1)+'summonerid']= match.participants[playernum].summoner_id
                matchdict['player'+str(playernum+1)+'summonername']= match.participants[playernum].summoner_name
                matchdict['player'+str(playernum+1)+'lane']= matchdata.participants[playernum].timeline.lane
                matchdict['player'+str(playernum+1)+'role']= matchdata.participants[playernum].timeline.role
                summoner_scrape_add(match.participants[playernum].summoner_id, match.participants[playernum].summoner_name)
        matchdict['MatchID']= matchid
        matchdict['matchtime']= matchtimestamp
        matchdict['patch_version'] = matchversion
        matchdict['team1_win']= team1win
        matchdict['team2_win']= team2win
        #print(matchdict)
        singlematch = pd.DataFrame([matchdict],index=[matchlist[matchiter].id])
        matchesdf= pd.concat([matchesdf,singlematch])
        #print(matchesdf.head())
        print('matches'+str(matchesdf.shape))
        print('Summoners'+str(summoners_to_scrape.shape))
    matchesdf.to_csv('matches.csv')
    summoners_to_scrape.to_csv('scrapetargets.csv')

new match... scraping..
scraping 2544164782
matches(2799, 55)
Summoners(4091, 4)
new match... scraping..
scraping 2544143352
matches(2800, 55)
Summoners(4091, 4)
new match... scraping..
scraping 2543632928
matches(2801, 55)
Summoners(4092, 4)
new match... scraping..
scraping 2543620116
matches(2802, 55)
Summoners(4094, 4)
new match... scraping..
scraping 2542862443
matches(2803, 55)
Summoners(4095, 4)
new match... scraping..
scraping 2542849804
matches(2804, 55)
Summoners(4100, 4)
new match... scraping..
scraping 2542806349
matches(2805, 55)
Summoners(4104, 4)
new match... scraping..
scraping 2542784343
matches(2806, 55)
Summoners(4106, 4)
new match... scraping..
scraping 2542729116
matches(2807, 55)
Summoners(4106, 4)
new match... scraping..
scraping 2542720983
matches(2808, 55)
Summoners(4111, 4)
new match... scraping..
scraping 2542251589
matches(2809, 55)
Summoners(4113, 4)
new match... scraping..
scraping 2542179300
matches(2810, 55)
Summoners(4116, 4)
new match... scraping..
scra

scraping 2505861054
matches(2900, 55)
Summoners(4222, 4)
matchfound, on to the next
new match... scraping..
scraping 2505580240
matches(2901, 55)
Summoners(4223, 4)
new match... scraping..
scraping 2504090233
matches(2902, 55)
Summoners(4223, 4)
new match... scraping..
scraping 2504011846
matches(2903, 55)
Summoners(4223, 4)
new match... scraping..
scraping 2503849623
matches(2904, 55)
Summoners(4224, 4)
new match... scraping..
scraping 2503847590
matches(2905, 55)
Summoners(4224, 4)
new match... scraping..
scraping 2503836476
matches(2906, 55)
Summoners(4225, 4)
new match... scraping..
scraping 2503834163
matches(2907, 55)
Summoners(4226, 4)
new match... scraping..
scraping 2503244061
matches(2908, 55)
Summoners(4226, 4)
new match... scraping..
scraping 2503231259
matches(2909, 55)
Summoners(4226, 4)
new match... scraping..
scraping 2501889211
matches(2910, 55)
Summoners(4226, 4)
new match... scraping..
scraping 2501877747
matches(2911, 55)
Summoners(4228, 4)
new match... scraping..
s

scraping 2466307733
matches(3000, 55)
Summoners(4315, 4)
new match... scraping..
scraping 2466286592
matches(3001, 55)
Summoners(4316, 4)
new match... scraping..
scraping 2466050929
matches(3002, 55)
Summoners(4317, 4)
new match... scraping..
scraping 2466005433
matches(3003, 55)
Summoners(4317, 4)
new match... scraping..
scraping 2465949169
matches(3004, 55)
Summoners(4317, 4)
new match... scraping..
scraping 2465934103
matches(3005, 55)
Summoners(4319, 4)
new match... scraping..
scraping 2465878757
matches(3006, 55)
Summoners(4319, 4)
new match... scraping..
scraping 2463545966
matches(3007, 55)
Summoners(4320, 4)
new match... scraping..
scraping 2463540859
matches(3008, 55)
Summoners(4320, 4)
new match... scraping..
scraping 2463484988
matches(3009, 55)
Summoners(4320, 4)
new match... scraping..
scraping 2463438083
matches(3010, 55)
Summoners(4321, 4)
new match... scraping..
scraping 2463443849
matches(3011, 55)
Summoners(4321, 4)
new match... scraping..
scraping 2463419829
matches(

scraping 2440549830
matches(3100, 55)
Summoners(4428, 4)
new match... scraping..
scraping 2440372345
matches(3101, 55)
Summoners(4431, 4)
new match... scraping..
scraping 2440360638
matches(3102, 55)
Summoners(4437, 4)
new match... scraping..
scraping 2440318555
matches(3103, 55)
Summoners(4439, 4)
new match... scraping..
scraping 2440335224
matches(3104, 55)
Summoners(4441, 4)
new match... scraping..
scraping 2440002429
matches(3105, 55)
Summoners(4445, 4)
new match... scraping..
scraping 2434297087
matches(3106, 55)
Summoners(4449, 4)
new match... scraping..
scraping 2434282762
matches(3107, 55)
Summoners(4452, 4)
new match... scraping..
scraping 2434257012
matches(3108, 55)
Summoners(4454, 4)
new match... scraping..
scraping 2434199590
matches(3109, 55)
Summoners(4459, 4)
new match... scraping..
scraping 2432834006
matches(3110, 55)
Summoners(4464, 4)
new match... scraping..
scraping 2432811268
matches(3111, 55)
Summoners(4467, 4)
new match... scraping..
scraping 2432758097
matches(

scraping 2405348245
matches(3201, 55)
Summoners(4845, 4)
new match... scraping..
scraping 2405325033
matches(3202, 55)
Summoners(4847, 4)
new match... scraping..
scraping 2405268209
matches(3203, 55)
Summoners(4852, 4)
new match... scraping..
scraping 2404291528
matches(3204, 55)
Summoners(4861, 4)
new match... scraping..
scraping 2404137795
matches(3205, 55)
Summoners(4868, 4)
new match... scraping..
scraping 2403858929
matches(3206, 55)
Summoners(4872, 4)
new match... scraping..
scraping 2403845326
matches(3207, 55)
Summoners(4878, 4)
new match... scraping..
scraping 2403789473
matches(3208, 55)
Summoners(4886, 4)
new match... scraping..
scraping 2402052210
matches(3209, 55)
Summoners(4891, 4)
new match... scraping..
scraping 2402028044
matches(3210, 55)
Summoners(4896, 4)
new match... scraping..
scraping 2402016592
matches(3211, 55)
Summoners(4902, 4)
new match... scraping..
scraping 2400932949
matches(3212, 55)
Summoners(4909, 4)
new match... scraping..
scraping 2400427491
matches(

...

new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scr

new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scr

new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scr

new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scraping..
new match... scr

KeyboardInterrupt: 

In [11]:
matchesdf2 = matchesdf.tail()

In [13]:
matchesdf2.dropna(axis=1, inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [75]:
matchesdf.to_csv('matches.csv')

In [89]:
matchesdf = pd.read_csv('matches.csv', index_col='matchid')

In [115]:
summoners_to_scrape.shape

(20273, 4)

In [116]:
matchesdf.to_csv('matches.csv')
summoners_to_scrape.to_csv('scrapetargets.csv')

In [117]:
matchesdf.tail()

,MatchID,matchtime,patch_version,player10champid,player10lane,player10role,player10summonerid,player10summonername,player1champid,player1lane,...,player8role,player8summonerid,player8summonername,player9champid,player9lane,player9role,player9summonerid,player9summonername,team1_win,team2_win
2385961967,2.385962e+09,2016-12-29 18:06:33.740000,6.24.169.4424,57.0,TOP,SOLO,25880323.0,impala tamer,412.0,BOTTOM,...,DUO_CARRY,19215996.0,Ûzì,267.0,BOTTOM,DUO_SUPPORT,61281860.0,Marigold,True,False
2385771764,2.385772e+09,2016-12-29 06:55:09.866000,6.24.169.4424,201.0,BOTTOM,DUO_SUPPORT,35520324.0,DOPEDlCK,51.0,BOTTOM,...,SOLO,48655696.0,2019,82.0,TOP,SOLO,24392633.0,Lyxical,True,False
2385734811,2.385735e+09,2016-12-29 06:10:19.717000,6.24.169.4424,40.0,BOTTOM,DUO,46579743.0,Gotta Go Fayst,64.0,JUNGLE,...,SOLO,380646.0,Unholy Wish,54.0,TOP,SOLO,48321584.0,Tahm K,False,True
2385711877,2.385712e+09,2016-12-29 05:37:40.577000,6.24.169.4424,421.0,JUNGLE,NONE,25793372.0,Asesh,432.0,BOTTOM,...,DUO_SUPPORT,48321584.0,Tahm K,55.0,MIDDLE,SOLO,30314638.0,DarkWingsForSale,False,True
2385664234,2.385664e+09,2016-12-29 04:46:00.734000,6.24.169.4424,122.0,TOP,SOLO,38625075.0,jaxe40,22.0,BOTTOM,...,NONE,31296001.0,09 12 2015,55.0,MIDDLE,SOLO,83089382.0,Agente Schrader,False,True
